In [1]:
import pandas as pd
import numpy as np
import pickle, io
from nltk.stem import WordNetLemmatizer

from google.cloud import vision
from google.cloud.vision import types
from oauth2client.client import GoogleCredentials

In [2]:
clf,vectorizer= pickle.load(open('CompostClassifier.pkl','rb'))

credentials = GoogleCredentials.get_application_default()
client = vision.ImageAnnotatorClient()

In [4]:
def get_lemma(word,lemmatizer=WordNetLemmatizer()):
    _word = word.split(' ')
    for i,_w in enumerate(_word):
        _word[i] = lemmatizer.lemmatize(_w.lower())
    return ' '.join(_word)

def get_vect(x,vectorizer):
    return vectorizer.transform(x).toarray()

def get_class_weights(word,clf=clf,vectorizer=vectorizer):
    return clf.predict_proba(get_vect(np.array([get_lemma(word)]),vectorizer))

def get_weighted_class(vision,clf=clf,vectorizer=vectorizer):
    i,x = 0,[]
    for key,value in vision.items():
        if i==0:
            if clf.predict(get_vect(np.array([get_lemma(key)]),vectorizer))==0:
                return 0
            else:
                i=1
        x.append(value*get_class_weights(key,clf,vectorizer).T)
    x = np.array(x).sum(axis=0)
    return np.argmax(x/x.sum())


def get_labels(filename,client=client):
    with io.open(filename, 'rb') as image_file:
        content=image_file.read()
    image = types.Image(content=content)
    response = client.label_detection(image=image)
    labels,scores = [],[]
    for label in response.label_annotations:
        labels.append(label.description)
        scores.append(label.score)
    return dict(zip(labels,scores))

def best_guess(vision,clf=clf,vectorizer=vectorizer):
    pred = get_weighted_class(vision)
    if pred==0:
        col = 'red'
        rec = 'CompostMeNot :-('
        msg = 'Remember: anything labeled “Compostable” or “#7 PLA” goes to the food and yard waste.'
    else:
        col = 'green'
        rec = 'CompostMe!!! Yaaaay!!!!!!'
        msg = 'Remember: dry paper/cardboard products go to recycle, wet/greasy/soiled ones to compost.'
    return col,rec,msg

def DoAll(filename):
    vision = get_labels(filename)
    col,rec,msg = best_guess(vision)
    rec = '\n{:s}\n'.format(rec)
    return col,rec,msg

In [30]:
col,rec,msg = DoAll('optimization_images/024_leftover_dinner.png')

In [27]:
col

'green'

In [32]:
print(rec)


CompostMe!!! Yaaaay!!!!!!



In [14]:
filename = 'optimization_images/001_Chicken-Bones.jpg'
vision = get_labels(filename)
x = []
tmp = []
for key,value in vision.items():
    tmp.append(get_class_weights(key,clf,vectorizer).T)
    x.append(value*get_class_weights(key,clf,vectorizer).T)

In [15]:
print(vision)

{'Food': 0.9541228413581848, 'Dish': 0.9422977566719055, 'Cuisine': 0.9349826574325562, 'Meat': 0.8480419516563416, 'Ingredient': 0.8450334072113037, 'Pulled pork': 0.842769205570221, 'Kalua': 0.7459040284156799, 'Carnitas': 0.6611395478248596, 'Recipe': 0.600089430809021, 'Produce': 0.592475175857544}


In [16]:
print(tmp)

[array([[0.02615632],
       [0.97384368]]), array([[0.45048515],
       [0.54951485]]), array([[0.32996346],
       [0.67003654]]), array([[0.30526593],
       [0.69473407]]), array([[0.32996346],
       [0.67003654]]), array([[0.31015381],
       [0.68984619]]), array([[0.32996346],
       [0.67003654]]), array([[0.31687744],
       [0.68312256]]), array([[0.32996346],
       [0.67003654]]), array([[0.39121165],
       [0.60878835]])]


In [17]:
print(x)

[array([[0.02495635],
       [0.92916649]]), array([[0.42449115],
       [0.51780661]]), array([[0.30851011],
       [0.62647255]]), array([[0.25887831],
       [0.58916364]]), array([[0.27883014],
       [0.56620326]]), array([[0.26138808],
       [0.58138113]]), array([[0.24612107],
       [0.49978296]]), array([[0.20950021],
       [0.45163934]]), array([[0.19800758],
       [0.40208185]]), array([[0.23178319],
       [0.36069199]])]


In [19]:
x = np.array(x).sum(axis=0)
print(x/x.sum())

[[0.30657843]
 [0.69342157]]
